In [ ]:
!unzip dataset.zip

In [ ]:
!pip install sympy==1.11.1  #only and only if the imports dont work even after restarting runtime, otherwise skip this.

In [2]:
!nvidia-smi                 #Free T4 GPU runtime

Sun Sep 21 10:31:46 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   34C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

# Training phase

In [ ]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import time
import torch.nn as nn
import torch.optim as optim
from PIL import Image

In [ ]:
img_transformer = transforms.Compose([
    transforms.Resize((128, 128)),   # Resize to uniform size
    transforms.ToTensor(),           # Convert to tensor
])

In [ ]:
dataset = datasets.ImageFolder(root="dataset", transform=img_transformer)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)
print("\nDetected Classes:", dataset.classes)

In [ ]:

save_model_name = input("Save the model as: _______.pth: ")

l_r = 0.001
try:
    l_r = float(input("\n\033[34mLearning rate [default lr=0.001]\033[0m \nthis value controls how big a step the optimizer takes when updating model weights during training.\nLess the lr value more the training time, more the accuracy\nEnter the lr value or simply press enter to proceed with default value: "))
except:
    l_r = 0.001
try:
   epc = int(input("\n\033[34mEpochs [default: 10]\033[0m \n(An epoch is one complete pass through the entire training dataset by the model)\nEnter the number of epochs to execute or simply press enter to proceed with default value: "))
except:
   epc = 10


In [ ]:
class CardCNN(nn.Module):
    def __init__(self, num_classes,  colour_channels=3):
        super(CardCNN, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d( colour_channels, 32, kernel_size=3, padding=1),  # Input 3x128x128
            nn.ReLU(),
            nn.MaxPool2d(2),                             # 32x64x64

            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),                             # 64x32x32
        )
        self.fc = nn.Sequential(
            nn.Linear(64 * 32 * 32, 128),
            nn.ReLU(),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        x = self.conv(x)
        x = x.view(x.size(0), -1)
        return self.fc(x)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"\nUsing device: {f"\033[32m{device}\033[0m" if torch.cuda.is_available() else f"{device}"} \n")
model = CardCNN(num_classes=len(dataset.classes), colour_channels=3).to(device)      #default 3 (RGB), change to 1 for grayscale

criterion = nn.CrossEntropyLoss()
if l_r == "":
    optimizer = optim.Adam(model.parameters(), lr=0.001)
else:
    optimizer = optim.Adam(model.parameters(), lr=l_r)

In [ ]:
d =0
start_time = time.time()
print(f"\033[32mtraining started...\033[0m \ndetails:\n lr count: {l_r}\n epoch count: {epc}\n")

try:
    for epoch in range(epc):
        print(f"\033[33mEpoch: {epoch+1}\033[0m")
        t = time.time()
        total_loss = 0
        model.train()
        for images, labels in dataloader:

            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            print(f"  loss: {loss.item()}")
            total_loss += loss.item()

        print(f"Epoch - {epoch+1}:\n Total loss: {total_loss:.4f}")
        t = (time.time() - t)
        print(f" time taken for epoch {epoch+1}: {(t)} seconds")
        d +=t

    print(f"\n\033[32mTraining completed in {time.time() - start_time:.2f} seconds \nAverage time per epoch: {(d)/epc}\033[0m")

    torch.save(model.state_dict(), f"{save_model_name}.pth")
    print(f"model saved as: {save_model_name}.pth")
except Exception as e:
    print(f"error: {e}")

# Testing/Prediction

In [ ]:
#incase you didn't run training phase
import torch
import torch.nn as nn
from PIL import Image
from torchvision import datasets, transforms

In [ ]:
img_transformer = transforms.Compose([
    transforms.Resize((128, 128)),   # Resize to uniform size
    transforms.ToTensor(),           # Convert to tensor
])

In [ ]:
#classes = ['ace of clubs', 'ace of diamonds', 'ace of hearts', 'ace of spades', 'eight of clubs', 'eight of diamonds', 'eight of hearts', 'eight of spades', 'five of clubs', 'five of diamonds', 'five of hearts', 'five of spades', 'four of clubs', 'four of diamonds', 'four of hearts', 'four of spades', 'jack of clubs', 'jack of diamonds', 'jack of hearts', 'jack of spades', 'joker', 'king of clubs', 'king of diamonds', 'king of hearts', 'king of spades', 'nine of clubs', 'nine of diamonds', 'nine of hearts', 'nine of spades', 'queen of clubs', 'queen of diamonds', 'queen of hearts', 'queen of spades', 'seven of clubs', 'seven of diamonds', 'seven of hearts', 'seven of spades', 'six of clubs', 'six of diamonds', 'six of hearts', 'six of spades', 'ten of clubs', 'ten of diamonds', 'ten of hearts', 'ten of spades', 'three of clubs', 'three of diamonds', 'three of hearts', 'three of spades', 'two of clubs', 'two of diamonds', 'two of hearts', 'two of spades']
dataset = datasets.ImageFolder(root="dataset", transform=img_transformer)
# you get the point...
#print("Classes:", classes)
print("Classes:", dataset.classes)

In [ ]:
class CardCNN(nn.Module):                                   # Make sure this matches the original training model class
    def __init__(self, num_classes, colour_channels= 3):
        super(CardCNN, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(colour_channels, 32, kernel_size=3, padding=1),     # Input 3x128x128
            nn.ReLU(),
            nn.MaxPool2d(2),                                # 32x64x64

            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),                                # 64x32x32
        )
        self.fc = nn.Sequential(
            nn.Linear(64 * 32 * 32, 128),
            nn.ReLU(),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        x = self.conv(x)
        x = x.view(x.size(0), -1)
        return self.fc(x)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"\nUsing device: {f"\033[32m{device}\033[0m" if torch.cuda.is_available() else f"{device}"} \n")

model = CardCNN(num_classes=len(dataset.classes), colour_channels=3).to(device)           # Make sure this matches the original training model and change colour channels if you have to


In [ ]:

model.eval()  # Set model to evaluation mode
image = Image.open("test.png").convert("RGB")               # Replace with your image
image = img_transformer(image).unsqueeze(0)
with torch.no_grad():
    images.to(device)
    output = model(image)
    predicted_index = output.argmax(1).item()

print(f"\033[32mPredicted class: {dataset.classes[predicted_index]}\033[0m")

image = Image.open("test1.jpg").convert("RGB")               # Replace with your image
image = img_transformer(image).unsqueeze(0)
with torch.no_grad():
    output = model(image)
    predicted_index = output.argmax(1).item()
print(f"\033[32mPredicted class: {dataset.classes[predicted_index]}\033[0m")

# Model Evaluation Phase

In [ ]:
# transforms + dataset
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor()
])

test_dataset = datasets.ImageFolder(root = "dataset", transform=transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"\nUsing device: {f"\033[32m{device}\033[0m" if f"{device}"=="cuda" else f"{device}"} \n")


Using device: cuda 



In [ ]:
model = CardCNN(num_classes=len(test_dataset.classes), colour_channels=3)      # same architecture as training change colour channels if you have to


model.load_state_dict(torch.load("model1.pth", map_location=device))
model.to(device)
model.eval()
criterion = nn.CrossEntropyLoss()

In [ ]:
correct = 0
wrong = 0
total = 0
batch_no = 0
total_loss = 0
print("Evaluating accuracy of the mmodel...")
print(f"\nUsing device: {f"\033[32m{device}\033[0m" if f"{device}" == "cuda" else f"{device}"} \n")

start_time = time.time()
with torch.no_grad():                           # disables gradient calculation (faster, less memory)
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)    # get class with highest probability
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        loss = criterion(outputs, labels)
        total_loss += loss.item()
        batch_no+=1
        print(f"batch: {batch_no+1}")


print(f"\nTotal images: {total}\nCorrect predictions: {f"\033[32m{correct}\033[0m" if ((100 * correct / total) >=98.0)  else (f"\033[31m{correct}\033[0m" if((100 * correct / total)<=60.0) else f"{correct}")}  \nAverage loss (per image): {total_loss/total}")
print(f"Accuracy of the model: { f"\033[32m{(100 * correct / total)}%\033[0m" if ((100 * correct / total) >=98.0)  else (f"\033[31m{(100 * correct / total)}%\033[0m" if((100 * correct / total)<=60.0) else f"{(100 * correct / total)}%" )}")
print(f"\n\033[32mEvaluation completed in {time.time() - start_time:.2f} seconds\033[0m")